### Case Study: COVID-19 Evolution in Mexico City & Correlation with Proximity Retailing Activity

##### 0.0 - Section Description

insert description!!!

##### 0.1 - Libraries, API keys, Configuration & Data Loading



In [ ]:
#Libraries
import pandas as pd
import geopandas as gpd #GIS dependency for spatial data manipulation
import requests

#API keys
from api_keys import gkey